In [1]:
import pandas as pd

In [2]:
businesses_data_file_path = '../data/processed/businesses_with_boroughs.csv'
population_data_file_path = '../data/processed/population_by_borough.csv'

In [3]:
businesses_data = pd.read_csv(businesses_data_file_path)
population_data = pd.read_csv(population_data_file_path)

In [11]:
businesses_data_boroughs = businesses_data['area'].unique()
population_data_boroughs = population_data['Area Name'].unique()

In [16]:
overlapping_boroughs = set(businesses_data_boroughs).intersection(set(population_data_boroughs))
print(f'{len(overlapping_boroughs)} boroughs are present in both datasets (out of {len(businesses_data_boroughs)} in businesses data and {len(population_data_boroughs)} in population data):')

33 boroughs are present in both datasets (out of 34 in businesses data and 36 in population data):


In [17]:
non_overlapping_boroughs = set(businesses_data_boroughs).symmetric_difference(set(population_data_boroughs))
print(f'{len(non_overlapping_boroughs)} boroughs are not present in both datasets:')
for borough in non_overlapping_boroughs:
    print(borough)


4 boroughs are not present in both datasets:
Greater London
Outer London
nan
Inner London
